In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob

from pprint import pprint

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="../chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


벡터 저장소에 저장된 문서 수: 6


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=100,
)

messages=[
    ("system", "You are a heplful assistant"),
    ("user", "{query}")
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a heplful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [7]:
# 템플릿 입력 변수 출력
print(prompt.input_variables)

['query']


In [8]:
# input 값을 전달하여 프롬프트 렌더링
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")

print(prompt_text)

System: You are a heplful assistant
Human: 테슬라 창업자는 누구인가요?


In [9]:
# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)

# 모델의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프랙(Jeff Skoll), 이안 라이트(Ian Wright) 등 여러 명이 있습니다. 그러나 엘론 머스크가 가장 잘 알려진 인물로, 2004년에 테슬라에 투자하고 이후 CEO로서 회사를 이끌어


In [10]:
# LCEL 체인 구성
chain = prompt | llm

# 체인 출력
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a heplful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025FF2823D50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025FF28F19D0>, root_client=<openai.OpenAI object at 0x0000025FF21B0550>, root_async_client=<openai.AsyncOpenAI object at 0x0000025FF21D2B10>, model_name='gpt-4o-mini', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [11]:
# 체인의 입력 스키마 출력
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [12]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하고 회사의 비전을 이끌어가며 테슬라의 성장에 큰 기여를 했습니다.


In [ ]:
# 체인을 실행 - 옵션 2 => 전달하는 인자가 하나일 경우 key value 형태로 전달하지 않아도 됨
response = chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)가 아닙니다. 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타르펜닝(Mark Tarpenning)에 의해 설립되었습니다. 그러나 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다. 따라서 엘론 머스크는
